# Import Readmission raw data file and prepare analysis dataset

In [14]:
import swat as swat
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import getpass as gp
%matplotlib inline
plt.rcParams['figure.figsize'] = 6, 6

In [15]:
s = swat.CAS('localhost', 5570, authinfo='/home/sasdemo/.authinfo')

### Load the raw data into CAS

In [16]:
s.loadactionset(actionset="table")
if not s.table.tableExists(table="readmission_raw").exists:
    tbl = s.upload_file("/home/sasdemo/hls_hospital_readmission_model.csv", casout={"name":"readmission_raw"})

NOTE: Added action set 'table'.
NOTE: Cloud Analytic Services made the uploaded file available as table READMISSION_RAW in caslib CASUSER(sasdemo).
NOTE: The table READMISSION_RAW has been created in caslib CASUSER(sasdemo) from binary data uploaded to Cloud Analytic Services.


### Drop the partitioned and analysis datasets

In [46]:
if s.tableExists(caslib='public',table='readmission_partitioned').exists:
    print("Dropping table readmission_partitioned")
    s.table.droptable(caslib='public',table='readmission_partitioned')
if s.tableExists(caslib='public',table='readmission_analysis').exists:
    print("Dropping table readmission_analysis")
    s.table.droptable(caslib='public',table='readmission_analysis')

Dropping table readmission_partitioned
NOTE: Cloud Analytic Services dropped table readmission_partitioned from caslib Public.


### Create sample, partition variable for cross validation

In [47]:
s.loadactionset(actionset="sampling")

tbl.sampling.stratified(
   partInd=True,                                     
   samppct=10,                                       
   seed=1234,
   output={
      "casout":{"name":"readmission_partitioned", 
                "caslib":"public",
                "promote":True},
      "copyVars":"all"
   }
)

NOTE: Added action set 'sampling'.
NOTE: Using SEED=1234 for sampling.


[outputSize]

 {'outputNObs': 161067.0, 'outputNVars': 53}

[STRAFreq]

 Frequencies
 
    ByGrpID    NObs  NSamp
 0        0  161067  16107

[OutputCasTables]

    casLib                     Name Label    Rows  Columns  \
 0  Public  readmission_partitioned        161067       53   
 
                                             casTable  
 0  CASTable('readmission_partitioned', caslib='Pu...  

+ Elapsed: 0.222s, user: 0.639s, sys: 0.244s, mem: 60.9mb

### Prepare columns for diminsion reduction and imputation

In [48]:
sci = s.columninfo(table={"name":"readmission_partitioned",'caslib':"public"}).ColumnInfo.iloc[:,[0,3]]

In [49]:
var_double = sci[sci['Type'] == 'double'].iloc[:,0].tolist()
var_char = sci[sci['Type'] == 'varchar'].iloc[:,0].tolist()
drops = var_double + var_char

removevar = ["readmit_number",'ENCOUNTER_KEY','PATIENT_NUMBER','DOCTOR','MS_DRG_CODE',
                  'DRG_APR_CODE','DRG_APR_SEVERITY','DIAGNOSIS_SUBCAT_CODE','DIAGNOSIS_ICD_CODE',
                  'PROCEDURE_SUBCAT_CODE','PROCEDURE_ICD_CODE','_PartInd_','Disch_Nurse_ID','ZIP']
[var_double.remove(x) for x in removevar]

removevar=['ADMIT_DATE','DISCHARGE_DATE','readmit_date','readmit_discharge_date','readmit_days',
           'Diagnosis_Group','MS_DRG_DESC','DRG_APR_DESC','DIAGNOSIS_SUBCAT_DESC','DIAGNOSIS_LONG_DESC',
           'PROCEDURE_SUBCAT_DESC','PROCEDURE_LONG_DESC','DEPARTMENT','STATECODE','CITY','County_name',
           'DESYNPUF_ID','HOSPITAL','REGION']
[var_char.remove(x) for x in removevar]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

### Perform dimension reduction

In [50]:
s.loadactionset(actionset="varReduce")
results = s.varReduce.super(
    table={'caslib':'public','name':'readmission_partitioned'},      
    maxeffects=15,
    varexp=0.99,
    analysis='VAR',
    tech='CORR',
    maxsteps=15,
    BIC=True,
    classVars=[{"vars":var_char}],
    model={
        "depVars":[{"name":'readmit_number'}],
        "effects":[{"vars":var_double + var_char}]}
)

NOTE: Added action set 'varReduce'.


In [51]:
print(results.SelectedEffects)

Selected Variables

   Number             Variable      Type
0       1  order_total_charges  INTERVAL
1       2             dx_group     CLASS
2       3              race_cd     CLASS
3       4          admit_month  INTERVAL
4       5       order_set_used  INTERVAL
5       6        DISCHARGED_TO     CLASS
6       7       operationcount  INTERVAL
7       8           op_visits6  INTERVAL
8       9           PatientAge  INTERVAL


In [52]:
keeps = results.SelectedEffects.iloc[:,1].tolist() + ['ADMIT_DATE','DISCHARGE_DATE',"readmit_number",'ENCOUNTER_KEY','PATIENT_NUMBER','_PartInd_']

In [53]:
def dropcol(col):
    s.table.alterTable(caslib="public",name="readmission_partitioned",columns=[{"drop":True,"name":col}]) 
[drops.remove(x) for x in keeps]
[dropcol(x) for x in drops]
s.columninfo(table={'caslib':'public','name':'readmission_partitioned'})

,Column,Label,ID,Type,RawLength,FormattedLength,NFL,NFD
0,ENCOUNTER_KEY,,1,double,8,12,0,0
1,PATIENT_NUMBER,,2,double,8,12,0,0
2,ADMIT_DATE,,4,varchar,9,9,0,0
3,DISCHARGE_DATE,,5,varchar,9,9,0,0
4,DISCHARGED_TO,,30,varchar,24,24,0,0
5,admit_month,,35,double,8,12,0,0
6,dx_group,,38,varchar,69,69,0,0
7,order_set_used,,39,double,8,12,0,0
8,order_total_charges,,40,double,8,12,0,0
9,readmit_number,,41,double,8,12,0,0


In [54]:
s.table.save(caslib='public', 
             name='readmission_analysis', 
             replace=True,
             permission='PUBLICREAD',
             table={"caslib":"public","name":'readmission_partitioned'}
            )

NOTE: Cloud Analytic Services saved the file readmission_analysis.sashdat in caslib Public.


[caslib]

 'Public'

[name]

 'readmission_analysis.sashdat'

+ Elapsed: 0.554s, user: 0.158s, sys: 0.14s, mem: 58mb

In [60]:
s.table.loadtable(caslib='public',path='readmission_analysis.sashdat', casout={'caslib':'public','name':'readmission_analysis','promote':True})

NOTE: Cloud Analytic Services made the file readmission_analysis.sashdat available as table READMISSION_ANALYSIS in caslib public.


[caslib]

 'public'

[tableName]

 'READMISSION_ANALYSIS'

[casTable]

 CASTable('READMISSION_ANALYSIS', caslib='public')

+ Elapsed: 0.00262s, user: 0.001s, sys: 0.001s, mem: 0.67mb

In [13]:
s.terminate()